In [61]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import matplotlib.pyplot as plt
import torchvision
import csv
import glob
import cv2
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from random import shuffle
import glob
#from torchsummary import summary


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
%matplotlib inline
# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'train', 'test']


In [62]:

#classes = ('plane', 'car', 'bird', 'cat',
#          'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [63]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 50, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(50, 100, 7)
        self.pool2 = nn.MaxPool2d(2,2)
        
        self.fc1 = nn.Linear(100 * 12 * 12, 120)
        self.fc2 = nn.Linear(120, 100)
        self.fc3 = nn.Linear(100, 2)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 100 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 50, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(50, 100, kernel_size=(7, 7), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=14400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=2, bias=True)
)


In [64]:
'''
#alternate way to create a list of file name and labels

import numpy as np
import os
PATH = '../input/'
fnames = np.array([f'train/{f}' for f in sorted(os.listdir(f'{PATH}train'))])
labels = np.array([(0 if 'cat' in fname else 1) for fname in fnames])
print(fnames[0:100] , labels[0:100])
'''

"\n#alternate way to create a list of file name and labels\n\nimport numpy as np\nimport os\nPATH = '../input/'\nfnames = np.array([f'train/{f}' for f in sorted(os.listdir(f'{PATH}train'))])\nlabels = np.array([(0 if 'cat' in fname else 1) for fname in fnames])\nprint(fnames[0:100] , labels[0:100])\n"

In [65]:
    shuffle_data = True  # shuffle the addresses before saving
    cat_dog_train_path = '../input/train/*.jpg'
    # read addresses and labels from the 'train' folder
    addrs = glob.glob(cat_dog_train_path)
    labels = [ [1,0] if 'cat' in addr else [0,1] for addr in addrs]  # 1 = Cat, 0 = Dog
    # to shuffle data
    if shuffle_data:
        c = list(zip(addrs, labels))
        shuffle(c)
        addrs, labels = zip(*c)
        print(labels[0:10])
        
    # Divide the hata into 60% train, 20% validation, and 20% test
    train_addrs = addrs[0:int(0.6*len(addrs))]
    train_labels = labels[0:int(0.6*len(labels))]
    #train_addrs.size
    
    val_addrs = addrs[int(0.6*len(addrs)):int(0.8*len(addrs))]
    val_labels = labels[int(0.6*len(addrs)):int(0.8*len(addrs))]
    
    test_addrs = addrs[int(0.8*len(addrs)):]
    test_labels = labels[int(0.8*len(labels)):]
    
    

([0, 1], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0])


In [68]:
    # loop over train addresses
    train_data = []
    for i in range(len(train_addrs[:1000])):
        # print how many images are saved every 10 images
        if i % 1000 == 0 and i > 1:
            print ('Train data: {}/{}'.format(i, len(train_addrs)))
        # read an image and resize to (64, 64)
        # cv2 load images as BGR, convert it to RGB
        addr = train_addrs[i]
        img = cv2.imread(addr)
        img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        train_data.append([np.array(img), np.array(train_labels[i])])
    shuffle(train_data)
    np.save('train_data.npy', train_data)
    
    
    
    
    
     # loop over test addresses
    #creating test data
    test_data = []
    for i in range(len(test_addrs[:1000])):
        # print how many images are saved every 1000 images
        if i % 1000 == 0 and i > 1:
            print ('Test data: {}/{}'.format(i, len(test_addrs)))
        # read an image and resize to (64, 64)
        # cv2 load images as BGR, convert it to RGB
        addr = test_addrs[i]
        img = cv2.imread(addr)
        img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        test_data.append([np.array(img), np.array(labels[i])])
    shuffle(test_data)
    np.save('test_data.npy', test_data)
    
    
    
    # loop over val addresses
    val_data = []
    for i in range(len(val_addrs[:1000])):
        # print how many images are saved every 1000 images
        if i % 1000 == 0 and i > 1:
            print ('Val data: {}/{}'.format(i, len(val_addrs)))
        # read an image and resize to (64, 64)
        # cv2 load images as BGR, convert it to RGB
        addr = val_addrs[i]
        img = cv2.imread(addr)
        img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        val_data.append([np.array(img), np.array(labels[i])])
    shuffle(val_data)
    np.save('val_data.npy', val_data)
    #print(val_data[1])

In [69]:
from torch.autograd import Variable
X = np.array([i[0] for i in train_data]).reshape(-1,64,64,3)
X = Variable(torch.Tensor(X))
X = X.reshape(-1,64,64,3)
X = X.permute(0,3,1,2)
print(X.shape)
#Y = Variable(torch.Tensor(Y))

Y = np.array([i[1] for i in train_data])
target = Variable(torch.Tensor(Y))
target = target.type(torch.LongTensor)

print(target.shape)
#print(target)

torch.Size([1000, 3, 64, 64])
torch.Size([1000, 2])


In [70]:
criterian = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.0001, momentum = 0.9)

In [71]:
for epoch in range(100):
    running_loss  = 0.0
    optimizer.zero_grad() #zero the parameter gradients
    output = net(X)
    
    loss = criterian(output, torch.max(target, 1)[1])
    
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    print(epoch, ':', running_loss)

/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0 : 0.7065433263778687
1 : 0.7014573216438293
2 : 0.6957210898399353
3 : 0.6923933029174805
4 : 0.6914113759994507
5 : 0.6915519833564758
6 : 0.6919784545898438
7 : 0.6923094391822815
8 : 0.6924932599067688
9 : 0.6925349831581116
10 : 0.6924546957015991
11 : 0.6922893524169922
12 : 0.6920660138130188
13 : 0.6917970776557922
14 : 0.6914887428283691
15 : 0.6911299228668213
16 : 0.6907241344451904
17 : 0.6902555823326111
18 : 0.6897157430648804
19 : 0.6890643239021301
20 : 0.6882694959640503
21 : 0.6872979998588562
22 : 0.6861180663108826
23 : 0.6846662759780884
24 : 0.6828793287277222
25 : 0.6806977987289429
26 : 0.6781398057937622
27 : 0.675240159034729
28 : 0.672492265701294
29 : 0.6702548265457153
30 : 0.668377161026001
31 : 0.6659526228904724
32 : 0.6631215214729309
33 : 0.6611957550048828
34 : 0.6598941087722778
35 : 0.6580750942230225
36 : 0.6558840870857239
37 : 0.6541180610656738
38 : 0.652590274810791
39 : 0.6504700183868408
40 : 0.6479725241661072
41 : 0.6458150148391724
42 : 0

In [72]:
test = np.array([i[0] for i in test_data]).reshape(-1,64,64,3)
test = Variable(torch.Tensor(test))
test = test.reshape(-1,64,64,3)
test = test.permute(0,3,1,2)
print(test.shape)
#Y = Variable(torch.Tensor(Y))

tlabels = np.array([i[1] for i in test_data])
tlabels = Variable(torch.Tensor(tlabels))
tlabels = tlabels.type(torch.long)

print(tlabels.shape)
print(tlabels)

torch.Size([1000, 3, 64, 64])
torch.Size([1000, 2])
tensor([[1, 0],
        [0, 1],
        [0, 1],
        ...,
        [0, 1],
        [0, 1],
        [1, 0]])


In [75]:
correct = 0
total = 0
with torch.no_grad():
    for data in zip(X,target):
        images, labels = data
        images = images.reshape(1,3,64,64)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        #total += labels.size(0)
        if((predicted == 0 and labels[0] == 1) or (predicted == 1 and labels[1]==1) ):
            correct+=1
        #correct += (predicted == labels).sum().item()
        #print(outputs,labels)
total = X.shape[0]
print('Train accuracy of the network on the' + str(total) +  'train images: %f %%' % (
    100 * (correct*1.0) / total) )
print(correct, total)

/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train accuracy of the network on the1000train images: 75.500000 %
755 1000


In [77]:
correct = 0
total = 0
with torch.no_grad():
    for data in zip(test,tlabels):
        images, labels = data
        images = images.reshape(1,3,64,64)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        #total += labels.size(0)
        if((predicted == 0 and labels[0] == 1) or (predicted == 1 and labels[1]==1) ):
            correct += 1
            
total = test.shape[0]
print('Test accuracy of the network on the ' + str(total) +  ' test images: %f %%' % (
    100 * (correct*1.0) / total) )
print(correct, total)

/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Test accuracy of the network on the 1000 test images: 53.600000 %
536 1000
